### data ###

In [ ]:
import xarray as xr

In [ ]:
# xarray dataset
ds = xr.open_dataset('/home/cz2397/data/ssh-eddies/tracks_AVISO_DT2014_daily_web.nc', decode_cf=False)
ds

In [ ]:
# track number
ds.track

In [ ]:
# observation number
ds.n

In [ ]:
# julian date
ds.j1

In [ ]:
# cyclonic
ds.cyc

In [ ]:
# longitude
ds.lon

In [ ]:
# latitude
ds.lat

In [ ]:
# amplitude
ds.A

In [ ]:
# radius scale
ds.L

In [ ]:
# maximum circum-averaged speed
ds.U

### multi-index ###

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd

In [ ]:
df = ds.to_dataframe()

In [ ]:
df.head()

In [ ]:
# track number
tra_num = max(df.track)
tra_num

In [ ]:
# observation number
obs_num = len(df.n)
obs_num

In [ ]:
track = np.asarray(df.track)
n = np.asarray(df.n)

In [ ]:
# multi-index construction
arrays = [track, n]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['track-id', 'point-id'])

In [ ]:
j1 = np.asarray(df.j1)
cyc = np.asarray(df.cyc)
lon = np.asarray(df.lon)
lat = np.asarray(df.lat)
A = np.asarray(df.A)
L = np.asarray(df.L)
U = np.asarray(df.U)

In [ ]:
matrix = np.zeros((obs_num, 7))

In [ ]:
for i in tqdm(range(obs_num)):
    matrix[i][0] = j1[i]
    matrix[i][1] = cyc[i]
    matrix[i][2] = lon[i]
    matrix[i][3] = lat[i]
    matrix[i][4] = A[i]
    matrix[i][5] = L[i]
    matrix[i][6] = U[i]

In [ ]:
DF = pd.DataFrame(matrix, index=multi_index)
DF.columns = ['j1', 'cyc', 'lon', 'lat', 'A', 'L', 'U']

In [ ]:
DF.head()

### time ###

In [ ]:
import datetime, jdcal

In [ ]:
def jday_to_datetime(jday, refday=0):
    y, m, d, f = jdcal.jd2gcal(jday, refday)
    h = int(f*24)
    return pd.to_datetime(datetime.datetime(y, m, d, h))

In [ ]:
date_fix = DF.j1.apply(jday_to_datetime)

In [ ]:
DF.j1 = date_fix
DF = DF.rename(columns = {'j1': 'date'})

In [ ]:
DF.head()

### longitude ###

In [ ]:
# longitude range setting
# from -180 to 180
lon_fix_a1 = DF.where(DF.lon < 540).lon - 360
lon_fix_a2 = DF.where(DF.lon >= 540).lon - 720
lon_fix_a = lon_fix_a1
lon_fix_a = lon_fix_a.fillna(lon_fix_a2)
DF.lon = lon_fix_a

In [ ]:
DF.head()

In [ ]:
# longitude range setting
# from 0 to 360
lon_fix_b1 = DF.where(DF.lon < 0).lon + 360
lon_fix_b2 = DF.where(DF.lon >= 0).lon
lon_fix_b = lon_fix_b1
lon_fix_b = lon_fix_b.fillna(lon_fix_b2)
DF.lon = lon_fix_b

In [ ]:
DF.head()

### radius ###

In [ ]:
radius_fix = DF.L*1000
DF.L = radius_fix
DF = DF.rename(columns = {'L': 'radius'})

In [ ]:
DF.head()

### geojson-point ###

In [ ]:
df.head()

In [ ]:
count = df.track
count = count.value_counts(normalize=False, sort=True, ascending=True, bins=None, dropna=False).reindex(range(1, tra_num+1))
count = pd.DataFrame(count)
count.index.name = 'track-id'
count.columns = ['obs-num']

In [ ]:
count.head()

In [ ]:
lon = np.asarray(DF.lon)
lat = np.asarray(DF.lat)
point = np.asarray(count['obs-num'])

In [ ]:
# origin coordinates
lon_ori = np.zeros(tra_num)
lat_ori = np.zeros(tra_num)

In [ ]:
# termination coordinates
lon_ter = np.zeros(tra_num)
lat_ter = np.zeros(tra_num)

In [ ]:
c = 0
i = 0

while i < obs_num:
    c = int(c) + 1
    lon_ori[c-1] = lon[i]
    lat_ori[c-1] = lat[i]
    i = i + int(point[c-1])
    lon_ter[c-1] = lon[i-1]
    lat_ter[c-1] = lat[i-1]

In [ ]:
lon_ori = pd.DataFrame(lon_ori)
lat_ori = pd.DataFrame(lat_ori)
lon_ter = pd.DataFrame(lon_ter)
lat_ter = pd.DataFrame(lat_ter)

In [ ]:
# index construction
index = np.zeros(tra_num)
for i in range(tra_num):
    index[i] = i+1
index = pd.DataFrame(index)
index = index.astype(int)

In [ ]:
# start center
sta_cen = pd.concat([index, lon_ori, lat_ori], axis=1)
sta_cen.columns = ['track-id', 'lon', 'lat']
sta_cen = sta_cen.set_index('track-id')

In [ ]:
sta_cen.head()

In [ ]:
# end center
end_cen = pd.concat([index, lon_ter, lat_ter], axis=1)
end_cen.columns = ['track-id', 'lon', 'lat']
end_cen = end_cen.set_index('track-id')

In [ ]:
end_cen.head()

### geojson-linestring ###

In [ ]:
lin_str = np.zeros((obs_num, 2))

In [ ]:
lon = np.asarray(DF.lon)
lat = np.asarray(DF.lat)

In [ ]:
for i in range(obs_num):
    lin_str[i][0] = lon[i]
    lin_str[i][1] = lat[i]

In [ ]:
lin_str = pd.DataFrame(lin_str, index=multi_index, columns=['lon', 'lat'])

In [ ]:
lin_str.head()

### geojson-polygon ###

In [ ]:
from numpy import cos, pi, sin

In [ ]:
radius = np.asarray(DF.radius)
point = np.asarray(count['obs-num'])

In [ ]:
rad_ori = np.zeros(tra_num)
rad_ter = np.zeros(tra_num)

In [ ]:
c = 0
i = 0

while i < obs_num:
    c = int(c) + 1
    rad_ori[c-1] = radius[i]
    i = i + int(point[c-1])
    rad_ter[c-1] = radius[i-1]

In [ ]:
rad_ori = pd.DataFrame(rad_ori)
rad_ter = pd.DataFrame(rad_ter)

In [ ]:
# start circle
sta_cir = pd.concat([index, lon_ori, lat_ori, rad_ori], axis=1)
sta_cir.columns = ['track-id', 'lon', 'lat', 'radius']
sta_cir = sta_cir.set_index('track-id')

In [ ]:
sta_cir.head()

In [ ]:
# end circle
end_cir = pd.concat([index, lon_ter, lat_ter, rad_ter], axis=1)
end_cir.columns = ['track-id', 'lon', 'lat', 'radius']
end_cir = end_cir.set_index('track-id')

In [ ]:
end_cir.head()

In [ ]:
# circle center number
cen_num = tra_num

In [ ]:
# circle arc number
arc_num = tra_num*33

In [ ]:
center = np.zeros(arc_num)
arc = np.zeros(arc_num)

In [ ]:
c = 1
i = 0
j = 1

while i < arc_num:
    while c <= 33:
        center[i] = j
        i = i+1
        c = c+1
    j = j+1
    c = 1

In [ ]:
center = center.astype(int)

In [ ]:
c = 1
i = 0
j = 1

while i < arc_num:
    while c <= 33:
        arc[i] = j
        i = i+1
        j = j+1
        c = c+1
    j = 1
    c = 1

In [ ]:
arc = arc.astype(int)

In [ ]:
# multi-index construction
arrays = [center, arc]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['center', 'arc'])

In [ ]:
# earth radius in meters
R = 6371*1000

__start polygon__

In [ ]:
lon = np.asarray(sta_cir.lon)
lat = np.asarray(sta_cir.lat)
radius = np.asarray(sta_cir.radius)

In [ ]:
theta = np.zeros(cen_num)
x = np.zeros(cen_num)
y = np.zeros(cen_num)
r = np.zeros(cen_num)

In [ ]:
for i in range(cen_num):
    theta[i] = lat[i]*(pi/180)
    r[i] = R*cos(theta[i])
    x[i] = (radius[i]/r[i])*(180/pi)
    y[i] = (radius[i]/R)*(180/pi)

In [ ]:
lon_sta_pol = np.zeros(arc_num)
lat_sta_pol = np.zeros(arc_num)
x_sta_pol = np.zeros(arc_num)
y_sta_pol = np.zeros(arc_num)

In [ ]:
c = 0
i = 0
j = 1

while i < arc_num:
    while j <= 33:
        lon_sta_pol[i] = lon[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < arc_num:
    while j <= 33:
        lat_sta_pol[i] = lat[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < arc_num:
    while j <= 33:
        x_sta_pol[i] = x[c]*cos((j-1)*(pi/16))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < arc_num:
    while j <= 33:
        y_sta_pol[i] = y[c]*sin((j-1)*(pi/16))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
sta_pol = np.zeros((arc_num, 2))

In [ ]:
for i in range(arc_num):
    sta_pol[i][0] = lon_sta_pol[i]+x_sta_pol[i]
    sta_pol[i][1] = lat_sta_pol[i]+y_sta_pol[i]

In [ ]:
# start polygon
sta_pol = pd.DataFrame(sta_pol, index=multi_index)
sta_pol.columns = ['lon', 'lat']

In [ ]:
sta_pol.head()

__end polygon__

In [ ]:
lon = np.asarray(end_cir.lon)
lat = np.asarray(end_cir.lat)
radius = np.asarray(end_cir.radius)

In [ ]:
theta = np.zeros(cen_num)
r = np.zeros(cen_num)
x = np.zeros(cen_num)
y = np.zeros(cen_num)

In [ ]:
for i in range(cen_num):
    theta[i] = lat[i]*(pi/180)
    r[i] = R*cos(theta[i])
    x[i] = (radius[i]/r[i])*(180/pi)
    y[i] = (radius[i]/R)*(180/pi)

In [ ]:
lon_end_pol = np.zeros(arc_num)
lat_end_pol = np.zeros(arc_num)
x_end_pol = np.zeros(arc_num)
y_end_pol = np.zeros(arc_num)

In [ ]:
c = 0
i = 0
j = 1

while i < arc_num:
    while j <= 33:
        lon_end_pol[i] = lon[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < arc_num:
    while j <= 33:
        lat_end_pol[i] = lat[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < arc_num:
    while j <= 33:
        x_end_pol[i] = x[c]*cos((j-1)*(pi/16))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < arc_num:
    while j <= 33:
        y_end_pol[i] = y[c]*sin((j-1)*(pi/16))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
end_pol = np.zeros((arc_num, 2))

In [ ]:
for i in range(arc_num):
    end_pol[i][0] = lon_end_pol[i]+x_end_pol[i]
    end_pol[i][1] = lat_end_pol[i]+y_end_pol[i]

In [ ]:
# end polygon
end_pol = pd.DataFrame(end_pol, index=multi_index)
end_pol.columns = ['lon', 'lat']

In [ ]:
end_pol.head()

### id ###

In [ ]:
_id = pd.DataFrame(index)
_id = pd.concat([index, index], axis=1)
_id.columns = ['track-id', 'id']
_id = _id.set_index('track-id')

In [ ]:
_id.head()

### date ###

In [ ]:
df.head()

In [ ]:
j1 = np.asarray(df.j1)

In [ ]:
j1_ori = np.zeros(tra_num)
j1_ter = np.zeros(tra_num)

In [ ]:
c = 0
i = 0

while i < obs_num:
    c = int(c) + 1
    j1_ori[c-1] = j1[i]
    i = i + int(point[c-1])
    j1_ter[c-1] = j1[i-1]

In [ ]:
j1_ori = pd.DataFrame(j1_ori)
j1_ter = pd.DataFrame(j1_ter)

In [ ]:
# start julian date
sta_jul = pd.concat([index, j1_ori], axis=1)
sta_jul.columns = ['track-id', 'j1']
sta_jul = sta_jul.set_index('track-id')

In [ ]:
sta_jul.head()

In [ ]:
# end julian date
end_jul = pd.concat([index, j1_ter], axis=1)
end_jul.columns = ['track-id', 'j1']
end_jul = end_jul.set_index('track-id')

In [ ]:
end_jul.head()

In [ ]:
# start date
sta_dat = sta_jul.j1.apply(jday_to_datetime)
sta_dat = pd.DataFrame(sta_dat)
sta_dat = sta_dat.rename(columns = {'j1': 'date'})

In [ ]:
sta_dat.head()

In [ ]:
# end date
end_dat = end_jul.j1.apply(jday_to_datetime)
end_dat = pd.DataFrame(end_dat)
end_dat = end_dat.rename(columns = {'j1': 'date'})

In [ ]:
end_dat.head()

### duration ###

In [ ]:
# duration in days
dur_day = end_dat - sta_dat
dur_day = dur_day.rename(columns = {'date': 'duration'})

In [ ]:
dur_day.head()

In [ ]:
# duration in integers
dur_int = np.zeros(tra_num)

In [ ]:
for i in range(tra_num):
    dur_int[i] = np.timedelta64(dur_day.duration[i+1], 'D')/np.timedelta64(1, 'D')

In [ ]:
dur_int = pd.DataFrame(dur_int)

In [ ]:
dur_int = pd.concat([index, dur_int], axis=1)
dur_int.columns = ['track-id', 'duration-in-days']
dur_int = dur_int.set_index('track-id')
dur_int['duration-in-days'] = dur_int['duration-in-days'].astype(int)

In [ ]:
dur_int.head()

### area ###

In [ ]:
df.head()

In [ ]:
L = np.asarray(ds.L)
radius = L*1000

In [ ]:
# circle area calculation
area = pi*(radius**2)

In [ ]:
sta_area = np.zeros(tra_num)

In [ ]:
i = 0
c = 0

while i < obs_num:
    c = int(c) + 1
    sta_area[c-1] = area[i]
    i = i + int(point[c-1])

In [ ]:
sta_area = pd.DataFrame(sta_area)

In [ ]:
# area in square meters
sta_area = pd.concat([index, sta_area], axis=1)
sta_area.columns = ['track-id', 'start-area']
sta_area = sta_area.set_index('track-id')

In [ ]:
sta_area.head()

### vorticity ###

In [ ]:
# relative vorticity
zeta = np.asarray((df.U*0.01)/(df.L*1000))

In [ ]:
# multi-index construction
arrays = [track, n]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['track-id', 'point-id'])

In [ ]:
zeta = pd.DataFrame(zeta, index=multi_index)
zeta.columns = ['relative-vorticity']

In [ ]:
zeta.head()

In [ ]:
lav = np.zeros(tra_num)

In [ ]:
for i in tqdm(range(tra_num)):
    lav[i] = zeta.loc[i+1].mean()

In [ ]:
lav = pd.DataFrame(lav)

In [ ]:
lav = pd.concat([index, lav], axis=1)
lav.columns = ['track-id', 'relative-vorticity']
lav = lav.set_index('track-id')

In [ ]:
lav.head()

### GeoJSON ###

In [ ]:
from geojson import LineString, Point, Polygon
from pymongo import MongoClient

In [ ]:
# collection
ssh_eddies_2016 = MongoClient().eddies.ssh_eddies_2016

In [ ]:
# remove documents from the collection
ssh_eddies_2016.remove()

In [ ]:
# insert documents into the collection
for i in tqdm(range(1, tra_num+1)):

    # geojson
    eddy = {
        '_id': int(_id.loc[i]),
        'date_start': sta_dat.loc[i]['date'],
        'date_end': end_dat.loc[i]['date'],
        'duration': dur_int.loc[i]['duration-in-days'],
        'loc_start': [sta_cen.loc[i]['lon'], sta_cen.loc[i]['lat']],
        'loc_end': [end_cen.loc[i]['lon'], end_cen.loc[i]['lat']],
        'area': sta_area.loc[i]['start-area'],
        'lav': lav.loc[i]['relative-vorticity'],
        'type': 'FeatureSet',
        'features': [
            {
                'type': 'Feature',
                'properties': {'name': 'start_center'},
                'geometry': Point(tuple(sta_cen.loc[i][['lon', 'lat']].values))
            },
            {
                'type': 'Feature',
                'properties': {'name': 'end_center'},
                'geometry': Point(tuple(end_cen.loc[i][['lon', 'lat']].values))
            },
            {
                'type': 'Feature',
                'properties': {'name': 'trajectory'},
                'geometry': LineString([tuple(x) for x in lin_str.loc[i][['lon', 'lat']].values])
            },
            {
                'type': 'Feature',
                'properties': {'name': 'start_polygon'},
                'geometry': Polygon([[tuple(x) for x in sta_pol.loc[i][['lon', 'lat']].values]])
            },
            {
                'type': 'Feature',
                'properties': {'name': 'end_polygon'},
                'geometry': Polygon([[tuple(x) for x in end_pol.loc[i][['lon', 'lat']].values]])
            }
        ]    
    }
    
    # mongodb
    ssh_eddies_2016.insert_one(dict(eddy))